In [1]:
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import KFold
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn import grid_search
from sklearn.metrics import mean_squared_error, make_scorer
from xgboost import XGBRegressor

C:\Users\xt\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\xt\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
def mean_squared_error_(ground_truth, predictions):
    return mean_squared_error(ground_truth, predictions) ** 0.5

RMSE = make_scorer(mean_squared_error_, greater_is_better=False)

In [7]:
class Ensemble(object):
    def __init__(self, n_folds, stacker, base_models):
        self.n_folds = n_folds
        self.stacker = stacker
        self.base_models = base_models

    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)

        folds = list(KFold(len(y), n_folds=self.n_folds, shuffle=True, random_state=2016))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))

        for i, clf in enumerate(self.base_models):

            print('Fitting For Base Model #{0} / {1} ---'.format(i+1, len(self.base_models)))

            S_test_i = np.zeros((T.shape[0], len(folds)))

            for j, (train_idx, test_idx) in enumerate(folds):

                print('--- Fitting For Fold #{0} / {1} ---'.format(j+1, self.n_folds))

                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
                # y_holdout = y[test_idx]
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_holdout)[:]
                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict(T)[:]

                print('Elapsed: %s minutes ---' % round(((time.time() - start_time) / 60), 2))

            S_test[:, i] = S_test_i.mean(1)

            print('Elapsed: %s minutes ---' % round(((time.time() - start_time) / 60), 2))

        print('--- Base Models Trained: %s minutes ---' % round(((time.time() - start_time) / 60), 2))
        
        
        param_grid = {
            'n_estimators': [50,60,70,80,90,100,110,120],
            'learning_rate': [0.05],
            'subsample': [0.75]
        }
        grid = grid_search.GridSearchCV(estimator=self.stacker, param_grid=param_grid, n_jobs=1, cv=5, verbose=20, scoring=RMSE)
        grid.fit(S_train, y)

        try:
            print('Param grid:')
            print(param_grid)
            print('Best Params:')
            print(grid.best_params_)
            print('Best CV Score:')
            print(-grid.best_score_)
            print('Best estimator:')
            print(grid.best_estimator_)
            print(message)
        except:
            pass

        print('--- Stacker Trained: %s minutes ---' % round(((time.time() - start_time) / 60), 2))

        y_pred = grid.predict(S_test)[:]

        return y_pred

In [8]:
df_all = pd.read_csv('df_all.csv', encoding='ISO-8859-1', index_col=0)
num_train = 74067
df_train = df_all.iloc[:num_train]
#df_train = df_train.iloc[:round(len(df_train)/20)]
df_test = df_all.iloc[num_train:]
#df_test = df_test.iloc[:round(len(df_test)/20)]

id_test = df_test['id']
y_train = df_train['relevance'].values

cols_to_drop = ['id', 'relevance']
for col in cols_to_drop:
    try:
        df_train.drop(col, axis=1, inplace=True)
        df_test.drop(col, axis=1, inplace=True)
    except:
        continue

X_train = df_train[:]
X_test = df_test[:]

print('Number of Features: ', len(X_train.columns.tolist()))

C:\Users\xt\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\xt\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Number of Features:  126


In [9]:
base_models = [
    RandomForestRegressor(
        n_jobs=1, random_state=2016, verbose=1,
        n_estimators=500, max_features=12
    ),
    ExtraTreesRegressor(
        n_jobs=1, random_state=2016, verbose=1,
        n_estimators=500, max_features=12
    ),
    GradientBoostingRegressor(
        random_state=2016, verbose=1,
        n_estimators=500, max_features=12, max_depth=8,
        learning_rate=0.05, subsample=0.8
    ),
    XGBRegressor(
        seed=2016,
        n_estimators=200, max_depth=8,
        learning_rate=0.05, subsample=0.8, colsample_bytree=0.85
    )
]
ensemble = Ensemble(
    n_folds=5,
    stacker=GradientBoostingRegressor(
        random_state=2017, verbose=1
    ),
    base_models=base_models
)


In [10]:
start_time = time.time()
y_pred = ensemble.fit_predict(X=X_train, y=y_train, T=X_test)
for i in range(len(y_pred)):
    if y_pred[i] < 1.0:
        y_pred[i] = 1.0
    if y_pred[i] > 3.0:
        y_pred[i] = 3.0
#pd.DataFrame({'id': id_test, 'relevance': y_pred}).to_csv('submission_ensemble.csv', index=False)

print('--- Submission Generated: %s minutes ---' % round(((time.time() - start_time) / 60), 2))

Fitting For Base Model #1 / 4 ---
--- Fitting For Fold #1 / 5 ---


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed: 22.5min finished
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    8.3s finished
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.8min finished


Elapsed: 24.46 minutes ---
--- Fitting For Fold #2 / 5 ---


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed: 23.8min finished
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    9.1s finished
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  2.0min finished


Elapsed: 50.42 minutes ---
--- Fitting For Fold #3 / 5 ---


...

         1           0.2773           0.0084            9.25s
         2           0.2705           0.0075            9.05s
         3           0.2627           0.0067            8.94s
         4           0.2554           0.0063            8.81s
         5           0.2502           0.0056            8.72s
         6           0.2449           0.0050            8.69s
         7           0.2402           0.0046            8.57s
         8           0.2360           0.0041            8.45s
         9           0.2324           0.0038            8.37s
        10           0.2276           0.0035            8.23s
        20           0.2073           0.0012            6.96s
        30           0.1993           0.0005            5.86s
        40           0.1971           0.0002            4.89s
        50           0.1961           0.0000            3.67s
        60           0.1950           0.0000            2.42s
        70           0.1946          -0.0000            1.20s
        

[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:  2.5min remaining:    0.0s


         1           0.2754           0.0082            9.46s
         2           0.2669           0.0074           10.31s
         3           0.2611           0.0065           10.04s
         4           0.2526           0.0061            9.62s
         5           0.2469           0.0055            9.41s
         6           0.2418           0.0049            9.18s
         7           0.2378           0.0045            8.98s
         8           0.2322           0.0040            8.80s
         9           0.2306           0.0037            8.64s
        10           0.2258           0.0033            8.55s
        20           0.2057           0.0013            7.27s
        30           0.1966           0.0005            5.98s
        40           0.1949           0.0002            4.74s
        50           0.1934           0.0001            3.53s
        60           0.1930          -0.0000            2.35s
        70           0.1927           0.0000            1.17s
        

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  6.9min finished


         1           0.2780           0.0084           13.59s
         2           0.2687           0.0077           13.82s
         3           0.2623           0.0070           13.21s
         4           0.2553           0.0063           12.68s
         5           0.2497           0.0057           12.47s
         6           0.2454           0.0051           12.11s
         7           0.2405           0.0047           12.13s
         8           0.2362           0.0041           11.98s
         9           0.2325           0.0038           11.98s
        10           0.2290           0.0035           11.97s
        20           0.2080           0.0012           10.18s
        30           0.1997           0.0005            8.22s
        40           0.1973           0.0002            6.46s
        50           0.1953           0.0001            4.80s
        60           0.1955           0.0000            3.17s
        70           0.1942           0.0000            1.58s
        

In [12]:
import os
from sklearn.metrics import mean_squared_error, make_scorer

def fmean_squared_error(ground_truth, predictions):
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.5
    return fmean_squared_error_

df_score = pd.read_csv(os.getcwd() + '/data/solution.csv', encoding="ISO-8859-1")
public_score = df_score[df_score.Usage == 'Public']
private_score = df_score[df_score.Usage == 'Private']

public_pred = pd.DataFrame({"id": list(id_test), "relevance": y_pred})
public_pred = public_pred[df_score.Usage == 'Public']
private_pred = pd.DataFrame({"id": list(id_test), "relevance": y_pred})
private_pred = private_pred[df_score.Usage == 'Private']

In [13]:
'public score: ',fmean_squared_error(list(public_pred.relevance), list(public_score.relevance)), 'private score: ',fmean_squared_error(list(private_pred.relevance), list(private_score.relevance))

('public score: ', 0.45183217401848913, 'private score: ', 0.45111671459846481)

In [14]:
y_pred

array([ 2.15673279,  2.12808402,  2.35362101, ...,  1.93183043,
        2.52306935,  2.69263093])